# LSLでデータにアクセスする

In [ ]:
from pylsl import StreamInfo, StreamInlet, resolve_streams
streams = resolve_streams(wait_time=3.)

In [ ]:
streams

# streamの情報を調べる

In [ ]:
inlet = StreamInlet(streams[0])

In [ ]:
print(inlet.info().name())                # inletに設定された名前
print(inlet.info().type())                # inletに設定された「タイプ」
print(inlet.info().channel_count())       # inletに含まれるチャンネルの数
print(inlet.info().nominal_srate())       # inletのサンプリングレート

In [ ]:
# チャンネル名を取得するための関数
def pick_ch_names(info):
    ch_xml = info.desc().child('channels').child('channel')
    ch_names = []
    for _ in range(info.channel_count()):
        ch_names.append(ch_xml.child_value('label'))
        ch_xml = ch_xml.next_sibling()
    return ch_names

In [ ]:
ch_names = pick_ch_names(inlet.info())
ch_names

In [ ]:
print(inlet.info().channel_format())
print(inlet.info().source_id()) 
print(inlet.info().version())
print(inlet.info().created_at())
print(inlet.info().uid())
print(inlet.info().session_id())
print(inlet.info().hostname())

In [ ]:
print(inlet.info().as_xml())

# 信号を受信する

## 必要な情報の入ったstreamを探す

In [ ]:
# stream名の一覧を作る
stream_names = []
for stream in streams:
    inlet = StreamInlet(stream)
    stream_names.append(inlet.info().name())

In [ ]:
stream_names

In [ ]:
import numpy as np
idx = np.where(np.array(stream_names)=='Pupil Primitive Data - Eye 0')[0][0]
inlet = StreamInlet(streams[idx])

In [ ]:
# 関数にまとめる
def inlet_specific_stream(stream_name):
    import numpy as np
    streams = resolve_streams(wait_time=3.)
    stream_names = []
    for stream in streams:
        inlet = StreamInlet(stream)
        stream_names.append(inlet.info().name())
    idx = np.where(np.array(stream_names)==stream_name)[0][0]
    inlet = StreamInlet(streams[idx])
    
    return inlet

In [ ]:
inlet = inlet_specific_stream('Pupil Primitive Data - Eye 0')

## サンプルを取得する

In [ ]:
# データのバッファ保存を開始する
inlet.open_stream()

In [ ]:
d, timestamps = inlet.pull_sample(timeout=1.)
print(d)

## チャンクを取得する

In [ ]:
### チャンクを取得する
d, timestamps = inlet.pull_chunk(timeout=1.)
print(np.array(d).shape)
print(d[0])

In [ ]:
### チャンクについて実験1
inlet = inlet_specific_stream('Pupil Primitive Data - Eye 0')
d, _ = inlet.pull_chunk()
print(np.array(d).shape)

## pull_chunk()の性質について

In [ ]:
### pullについて実験1
while True:                      # 条件を満たすまで何度でも繰り返す
    d, _ = inlet.pull_chunk()    # pullする
    d = np.array(d)              # 扱いやすいようnumpy.array形式にする
    print(d.shape)               # pullで引っ張ってきたデータのサイズを表示
    if d.shape[0]==0: break      # 引っ張ってきたデータが空っぽ（＝バッファが空）だったら終了

In [ ]:
### pullについての実験2
while True:                      # 条件を満たすまで何度でも繰り返す
    d, _ = inlet.pull_chunk()    # pullする
    d = np.array(d)              # 扱いやすいようnumpy.array形式にする
    if d.shape[0]==0: break      # 引っ張ってきたデータが空っぽ（＝バッファが空）だったら終了
    print(d[-1, 2])               # pullで引っ張ってきたデータ（のうち最も古いもの）のタイムスタンプを表示

In [ ]:
print(inlet.info().as_xml())

# リアルタイム解析してみる
Notebook上ではうまく動きません。  
実行する場合は、コピペで.pyファイルを作って走らせてください。

In [ ]:
from time import sleep
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
from pylsl import StreamInfo, StreamInlet, resolve_streams

def inlet_specific_stream(stream_name):
    import numpy as np
    streams = resolve_streams(wait_time=3.)
    stream_names = []
    for stream in streams:
        inlet = StreamInlet(stream)
        stream_names.append(inlet.info().name())
    idx = np.where(np.array(stream_names)==stream_name)[0][0]
    inlet = StreamInlet(streams[idx])
    return inlet

stream_name = 'Pupil Primitive Data - Eye 0'

fig = plt.figure()
ax = fig.add_subplot(111)

inlet = inlet_specific_stream(stream_name)
inlet.open_stream()    # バッファ開始
sleep(.1)               # バッファにある程度データをためる
print('START!')
while True:
    # データ取得
    if True:                                         # 正しい例
        d, _ = inlet.pull_chunk(max_samples=1024)    # バッファにあるデータを全部取る
        assert(len(d) < 1024)                        # 念のため、全部取り切れていることを確認する
        try:
            diameter = np.array(d)[-1, 0]            # とってきたデータの最後の部分を使う
        except:                                      # たまに何故かバッファが空になるので...
            pass                                     # その時はpassするようにごまかす

    if False:                                        # 悪い例（１）！！！
        d, _ = inlet.pull_chunk(max_samples=1)       # バッファから1サンプルだけ取得する
        diameter = np.array(d)[0, 0]

    if False:                                        # 悪い例（２）！！！
        d, _ = inlet.pull_sample(timeout=1.)         # pull_sampleを使って1サンプルだけ取得する
        diameter = d[0]

    # 取得した瞳孔サイズをリアルタイムに表示
    plt.cla()
    c = patches.Circle(xy=(0,0), radius=diameter/2)
    ax.add_patch(c)
    plt.xlim([-30, 30])
    plt.ylim([-30, 30])
    plt.pause(.1)